In [1]:
import tensorflow as tf
import numpy as np
import datetime
import json
import gzip
import matplotlib.pyplot as plt
import re
from tqdm import tqdm_notebook

In [2]:
sess = None

In [3]:
def reset_tf(sess = None, log_device_placement = False):
    if sess:
        sess.close()
    tf.reset_default_graph()
    tf.set_random_seed(0)
    return tf.InteractiveSession(config = tf.ConfigProto(log_device_placement = log_device_placement))

In [4]:
class HyperParameters:
    learning_rate = 1e-1
    
    vocab_size = 30000
    num_targets = 2000
    
    context_size = 81
    context_center_index = context_size // 2
    
    pipeline_batch_size = 64
    pipeline_num_parallel_calls = 4
    pipeline_prefetch_size = pipeline_batch_size * 16
    pipeline_shuffle_size = 5000

In [41]:
class EntityLinkingModel:
    def __init__(self, session, hp):
        self._session = session
        self._hp = hp
        
    def _parse_example(self, example_proto):
        parsed = tf.parse_single_example(example_proto, features = {
            'page_id': tf.FixedLenFeature([1], tf.int64),
            'target_id': tf.FixedLenFeature([1], tf.int64),
            'context_word_ids': tf.FixedLenFeature([self._hp.context_size], tf.int64) })
        
        return (
            parsed['target_id'],
            parsed['context_word_ids'])

    def _build_data_pipeline(self):
        with tf.variable_scope('dataset'):
            # placeholder: examples filenames
            self._dataset_filenames = tf.placeholder(tf.string, shape = [None])
            
            self._pipeline_limit = tf.placeholder_with_default(tf.constant(-1, dtype = tf.int64), [])

            # build examples dataset
            dataset = tf.data.TFRecordDataset(
                self._dataset_filenames,
                compression_type='GZIP')
            dataset = dataset.take(self._pipeline_limit)
            dataset = dataset.map(
                self._parse_example,
                num_parallel_calls = self._hp.pipeline_num_parallel_calls)
            dataset = dataset.shuffle(self._hp.pipeline_shuffle_size)
            dataset = dataset.prefetch(self._hp.pipeline_prefetch_size)
            dataset = dataset.batch(self._hp.pipeline_batch_size)

            # build dataset iterator
            self._dataset_iterator = dataset.make_initializable_iterator()
            (target_labels, context_word_ids) = self._dataset_iterator.get_next()
            target_labels = tf.squeeze(target_labels, axis = -1)

            # placeholders
            self._context_word_ids = tf.placeholder_with_default(
                context_word_ids,
                shape = [None, self._hp.context_size],
                name = 'context_word_ids')
            self._target_labels = tf.placeholder_with_default(
                target_labels,
                shape = [None],
                name = 'target_labels')
            
            self._minibatch_size = tf.shape(self._context_word_ids)[0]
            
    def _bincount_vectorized(self, A, dim):
        offsets = tf.expand_dims(tf.range(tf.shape(A)[0]) * dim, 1)
        A_ravel = tf.reshape(A + offsets, [-1])
        dim_ravel = tf.shape(A)[0] * dim
        bincount_ravel = tf.bincount(
            A_ravel, 
            minlength = dim_ravel,
            maxlength = dim_ravel)
        return tf.reshape(bincount_ravel, [-1, dim])            
            
    def _build_model(self):
        with tf.variable_scope('model'):
            features = self._bincount_vectorized(
                tf.cast(self._context_word_ids, tf.int32),
                self._hp.vocab_size)
            features = tf.cast(features, tf.float32)
            self._output_logits =  tf.layers.dense(features, self._hp.num_targets)
            
            self._features = features

    def _build_training_model(self):
        with tf.variable_scope('train'):
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = self._target_labels,
                logits = self._output_logits)
            
            self._total_loss = tf.reduce_sum(losses)
            self._mean_loss = tf.reduce_mean(losses)
            
            # N.B., tf.nn.softmax here is unnecessary?
            output_labels = tf.argmax(tf.nn.softmax(self._output_logits), axis=-1)
            self._num_correct_labels = tf.reduce_sum(tf.cast(
                tf.equal(output_labels, self._target_labels), 
                tf.int32))
            
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                self._global_step = tf.Variable(0, name='global_step', trainable=False)
                self._optimizer = tf.train.AdamOptimizer(learning_rate=self._hp.learning_rate)
                self._train_op = self._optimizer.minimize(
                    self._mean_loss,
                    global_step=self._global_step)
                
#                 # gradient clipping
#                 gradients, variables = zip(*self._optimizer.compute_gradients(self._mean_loss))
#                 gradients, _ = tf.clip_by_global_norm(
#                     gradients, 
#                     self._hp.gradient_clip_norm)
#                 self._train_op = self._optimizer.apply_gradients(
#                     zip(gradients, variables),
#                     global_step = self._global_step)

    
    def build_model(self):
        self._build_data_pipeline()
        self._build_model()
        self._build_training_model()

    def dump_statistics(self):
        total_parameters = 0
        for variable in tf.trainable_variables():
            # shape is an array of tf.Dimension
            shape = variable.get_shape()
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            print('parameters for "%s": %d' % (variable.name, variable_parameters))
            total_parameters += variable_parameters
        print('total parameters: %d' % total_parameters)

    def process(self,
                dataset_filename,
                header = 'results',
                train = False,
                show_progress = True,
                log_file = None,
                limit = -1):
        cum_loss = 0
        cum_num_examples = 0
        cum_correct_examples = 0
        
        start = datetime.datetime.now()

        self._session.run(self._dataset_iterator.initializer, feed_dict={
            self._dataset_filenames: [dataset_filename],
            self._pipeline_limit: limit
        })

        if show_progress:
            progress = tqdm_notebook(leave = False, desc = header)

        while True:
            try:
                (_,
                 curr_total_loss, 
                 curr_minibatch_size,
                 curr_num_correct_labels) = self._session.run(
                    (self._train_op if train else (),
                     self._total_loss,
                     self._minibatch_size,
                     self._num_correct_labels))
            except tf.errors.OutOfRangeError:
                break

            if show_progress:
                progress.update(curr_minibatch_size)

            cum_loss += curr_total_loss
            cum_num_examples += curr_minibatch_size
            cum_correct_examples += curr_num_correct_labels

        if show_progress:
            progress.close()
            
        finish = datetime.datetime.now()

        message = '%s (%d) (%s): loss=%g, accuracy=%g' % (
            header,
            tf.train.global_step(sess, self._global_step),
            finish - start,
            cum_loss / cum_num_examples,
            cum_correct_examples / cum_num_examples)
        print(message)
        if log_file:
            print(message, file=log_file)
            log_file.flush()

In [42]:
sess = reset_tf(sess)

model = EntityLinkingModel(sess, HyperParameters())
model.build_model()
model.dump_statistics()

parameters for "model/dense/kernel:0": 60000000
parameters for "model/dense/bias:0": 2000
total parameters: 60002000


In [57]:
sess.run(tf.global_variables_initializer())

In [58]:
for i in range(100):
    model.process(
        '../data/simplewiki/simplewiki-20171103.el_softmax_1.tiny.tfrecords.gz', 
        train = True,
        limit = 17)

HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

results (1) (0:00:00.078696): loss=7.65313, accuracy=0


HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

results (2) (0:00:00.067870): loss=1.04351, accuracy=0.411765


HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

...

results (98) (0:00:00.046522): loss=0.0866594, accuracy=0.941176


HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

results (99) (0:00:00.046341): loss=0.0866068, accuracy=0.941176


HBox(children=(IntProgress(value=1, bar_style='info', description='results', max=1), HTML(value='')))

results (100) (0:00:00.056267): loss=0.0865675, accuracy=0.941176
